In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [2]:
!wget https://s3-us-west-2.amazonaws.com/deepcognition/datasets/Soda_Bottles.zip

--2018-07-11 07:47:39--  https://s3-us-west-2.amazonaws.com/deepcognition/datasets/Soda_Bottles.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.200.208
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.200.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 293778648 (280M) [application/zip]
Saving to: ‘Soda_Bottles.zip’

Soda_Bottles.zip    100%[===================>] 280.17M  6.19MB/s    in 45s     

2018-07-11 07:48:25 (6.26 MB/s) - ‘Soda_Bottles.zip’ saved [293778648/293778648]



In [0]:
!cp -a Soda drive/soda

In [0]:
import os
os.chdir('drive/soda')

In [0]:
os.chdir('Soda Bottles/')

In [2]:
!ls

datalab  drive


In [0]:
!unzip Soda_Bottles.zip -d Soda

In [9]:
import os
os.chdir('-')

FileNotFoundError: ignored

In [8]:
!cd -

/bin/sh: 1: cd: can't cd to /tmp/tmp.Oz8ARMKTX6


In [0]:
#data = [[][][][][][][][]]
sum = 0
fold  = os.listdir('.')
fold.sort(key=str.lower)
print(fold)
for p in fold[2:-2]:
    sum = sum + len(os.listdir(p))
print(sum)        

['.DS_Store', '.ipynb_checkpoints', 'M.Beer', 'MD.Diet', 'MD.Orig', 'P.Cherry', 'P.diet', 'P.Orig', 'P.Rsugar', 'P.Zero', 'train.csv']
5791


In [0]:
import numpy as np
from PIL import Image

In [0]:
fold  = os.listdir('.')
fold.sort(key=str.lower)
print(fold[1:-1])
data = []
for p in fold[2:-2]:
    arr = []
    for i in os.listdir(p)[0:-100]:
        img = np.asarray(Image.open(p+'/'+i))
        arr.append(img)
    data.append(arr)

In [20]:
!ls

M.Beer	 MD.Orig    P.Cherry  P.Orig	P.Zero
MD.Diet  meta.json  P.diet    P.Rsugar	train.csv


In [0]:
import random

In [27]:
X1 = []
X2 = []
X3 = []
fold  = os.listdir('.')
fold.sort(key=str.lower)

for i in range(800):
    a,b = random.sample(range(0,8),2)
    c,d = random.sample(range(0,len(os.listdir((fold[1:-1])[a]))),2)
    e = np.random.randint(0,len(os.listdir((fold[1:-1])[b])))
    img = np.asarray(Image.open(fold[1:-1][a]+'/'+os.listdir((fold[1:-1])[a])[c]))
    X1.append(img)
    img = np.asarray(Image.open(fold[1:-1][a]+'/'+os.listdir((fold[1:-1])[a])[d]))
    X2.append(img)
    img = np.asarray(Image.open(fold[1:-1][b]+'/'+os.listdir((fold[1:-1])[b])[e]))
    X3.append(img)
X1 = np.asarray(X1)
X2 = np.asarray(X2)
X3 = np.asarray(X3)
print(X1.shape)

(800, 480, 640, 3)


In [8]:
import keras

Using TensorFlow backend.


In [9]:
X_input = keras.layers.Input((480,640,3))
X = keras.layers.Conv2D(3,(2,2),strides=(2,2),activation='relu')(X_input)
X = keras.layers.BatchNormalization()(X)
model = keras.applications.nasnet.NASNetMobile(input_shape=(240,320,3), include_top=False, weights='imagenet')
X = model(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.MaxPooling2D((8,10))(X)
X = keras.layers.Reshape((1,1056))(X)

m = keras.models.Model(inputs=X_input,outputs=X) 
m.summary()


19996672/19993200 [==============================] - 3s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 480, 640, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 240, 320, 3)       39        
_________________________________________________________________
batch_normalization_1 (Batch (None, 240, 320, 3)       12        
_________________________________________________________________
NASNet (Model)               (None, 8, 10, 1056)       4269716   
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 10, 1056)       4224      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 1056)        0         
_________________________________________________________________
reshape_1 (

In [9]:
m = keras.models.load_model('soda_siam.h5')

/usr/local/lib/python3.6/dist-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [10]:
X1_input = keras.layers.Input((480,640,3))
X2_input = keras.layers.Input((480,640,3))
X3_input = keras.layers.Input((480,640,3))

X_1 = m(X1_input)
X_2 = m(X2_input)
X_3 = m(X3_input)

X = keras.layers.Concatenate(axis=1)([X_1,X_2,X_3])

siam_model = keras.models.Model(inputs=[X1_input,X2_input,X3_input],outputs=X)
siam_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 480, 640, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 480, 640, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 480, 640, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1, 1056)      4273991     input_1[0][0]                    
                                                                 input_2[0][0]                    
          

In [0]:
import tensorflow as tf
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[:,0,:], y_pred[:,1,:], y_pred[:,2,:]
    anchor = tf.nn.l2_normalize(anchor,1)
    positive = tf.nn.l2_normalize(positive,1)
    negative = tf.nn.l2_normalize(negative,1)
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = 7*tf.losses.cosine_distance(anchor,positive,axis=1) + 3*tf.reduce_sum(tf.square(anchor - positive))
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = 7*tf.losses.cosine_distance(anchor,negative,axis=1) + 3*tf.reduce_sum(tf.square(anchor - negative))
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = pos_dist - neg_dist + alpha
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.maximum(basic_loss, 0)
    ### END CODE HERE ###
    
    return loss

In [0]:
siam_model.compile(optimizer='adam',loss=triplet_loss,metrics=["accuracy"])

In [0]:
y = np.zeros((X1.shape[0],3,1056))

In [29]:
info = siam_model.fit([X1,X2,X3],y,epochs=10,batch_size=16)

Epoch 1/10
800/800 [==============================] - 270s 338ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 2/10
800/800 [==============================] - 124s 155ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 3/10
800/800 [==============================] - 124s 155ms/step - loss: 0.0000e+00 - acc: 4.1667e-04
Epoch 4/10
800/800 [==============================] - 124s 155ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 5/10
800/800 [==============================] - 124s 155ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 6/10
496/800 [=================>............] - ETA: 47s - loss: 0.0000e+00 - acc: 6.7204e-04

800/800 [==============================] - 124s 155ms/step - loss: 0.0000e+00 - acc: 4.1667e-04
Epoch 7/10
800/800 [==============================] - 124s 155ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 8/10
800/800 [==============================] - 124s 155ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 9/10
800/800 [==============================] - 124s 155ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 10/10
800/800 [==============================] - 124s 155ms/step - loss: 0.0000e+00 - acc: 0.0000e+00


In [0]:
  m.save('soda_siam.h5')